In [23]:
!pip install torch_geometric torch gdown --quiet

In [2]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

Cloning into 'hackaton'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 81 (delta 4), reused 4 (delta 4), pack-reused 70 (from 1)
Receiving objects: 100% (81/81), 105.83 MiB | 41.41 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [24]:
%cd hackaton/

[Errno 2] No such file or directory: 'hackaton/'
/home/onyxia/work/DL-Hackathon/hackaton


In [4]:
!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets


Retrieving folder contents
Retrieving folder 1wcUVBNQkZ04zStXkglXSgERfIvjSHJiL A
Processing file 1C8sjkO6JS0j2SyVwQ07m8PhQ-pHpuI78 test.json.gz
Processing file 12N11n8gufNA_C1ns-1IeBseBHgrSfRI1 train.json.gz
Retrieving folder 1Tj5YoYYDDXjDxxi-cywZgoDkT0b1Qbz- B
Processing file 11GBlrXMdP3HSD60w-56Tu6rbGkR-Ifww test.json.gz
Processing file 13vp-Kwef3UgAwMG-dokGwKyARym9iqtL train.json.gz
Retrieving folder 1e3B_tBMd693Iwv8x3zRR9c47l5yt_5ey C
Processing file 18XVe65ZsQ0PDLCqQa4WmneVhyfjGcXmT test.json.gz
Processing file 1z5lvG2CytbLQZt7Jmo9BopzFd0pKejEj train.json.gz
Retrieving folder 1cvM0eZwpD4gzjo44_zdodxudVBMrLza1 D
Processing file 1Gna_dHnBLX8vKaYGAAqAbw5QPerrNK1u test.json.gz
Processing file 1Pc-6LMML80-AgEoLVs2Q5hLtmR_rTEek train.json.gz
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1C8sjkO6JS0j2SyVwQ07m8PhQ-pHpuI78
From (redirected): https://drive.google.com/uc?

In [25]:
!ls -lh datasets

total 16K
drwxr-sr-x 2 onyxia users 4.0K May 31 07:01 A
drwxr-sr-x 2 onyxia users 4.0K May 31 07:01 B
drwxr-sr-x 2 onyxia users 4.0K May 31 07:01 C
drwxr-sr-x 2 onyxia users 4.0K May 31 07:02 D


In [26]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

import torch.nn.functional as F

# Set the random seed
set_seed()


In [27]:
# try to use this one 
import gzip
import json
import torch
from torch_geometric.data import Dataset, Data
import os
from tqdm import tqdm 
from torch_geometric.loader import DataLoader

class GraphDataset(Dataset):
    def __init__(self, filename, transform=None, pre_transform=None):
        self.raw = filename
        self.graphs = self.loadGraphs(self.raw)
        super().__init__(None, transform, pre_transform)

    def len(self):
        return len(self.graphs)

    def get(self, idx):
        return self.graphs[idx]

    @staticmethod
    def loadGraphs(path):
        print(f"Loading graphs from {path}...")
        print("This may take a few minutes, please wait...")
        with gzip.open(path, "rt", encoding="utf-8") as f:
            graphs_dicts = json.load(f)
        graphs = []
        for graph_dict in tqdm(graphs_dicts, desc="Processing graphs", unit="graph"):
            graphs.append(dictToGraphObject(graph_dict))
        return graphs



def dictToGraphObject(graph_dict):
    edge_index = torch.tensor(graph_dict["edge_index"], dtype=torch.long)
    edge_attr = torch.tensor(graph_dict["edge_attr"], dtype=torch.float) if graph_dict["edge_attr"] else None
    num_nodes = graph_dict["num_nodes"]
    y = torch.tensor(graph_dict["y"][0], dtype=torch.long) if graph_dict["y"] is not None else None
    return Data(edge_index=edge_index, edge_attr=edge_attr, num_nodes=num_nodes, y=y)

In [28]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [29]:
class GCODLoss(torch.nn.Module):
    def __init__(self, gamma=0.2):
        super(GCODLoss, self).__init__()
        self.gamma = gamma

    def forward(self, logits, labels):
        ce_loss = F.cross_entropy(logits, labels, reduction='none')

        probs = F.softmax(logits, dim=1)
        true_probs = probs[range(len(labels)), labels]

        weight = (true_probs.detach() ** self.gamma)

        loss = weight * ce_loss
        return loss.mean()

In [30]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [31]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [32]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [33]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [34]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [35]:
def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['device'] = get_user_input("Which GPU to use if any", default=1, type_cast=int)
    args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin')
    args['drop_ratio'] = get_user_input("Dropout ratio", default=0.0, type_cast=float)
    args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['epochs'] = get_user_input("Number of epochs to train", default=10, type_cast=int)
    args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (Noisy CE)", default=1, type_cast=int)
    args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)

    
    return argparse.Namespace(**args)


# ABCD

In [37]:
gnn = 'gcn'
num_layer = 5
emb_dim = 300
drop_ratio = 0.5
baseline_mode = 2
num_checkpoints = 10


script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = num_checkpoints if num_checkpoints else 3
    
if gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=num_layer, emb_dim=emb_dim, drop_ratio=drop_ratio, virtual_node=False).to(device)
elif gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=num_layer, emb_dim=emb_dim, drop_ratio=drop_ratio, virtual_node=True).to(device)
elif gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=num_layer, emb_dim=emb_dim, drop_ratio=drop_ratio, virtual_node=False).to(device)
elif gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=num_layer, emb_dim=emb_dim, drop_ratio=drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if baseline_mode == 2:
    criterion = GCODLoss(gamma=0.2)
else:
    criterion = torch.nn.CrossEntropyLoss()

In [38]:
import os
import logging

# Nom spécial pour indiquer que c'est un pré-entraînement multi-dataset
pretrain_tag = "pretrain_ABCD"

# Dossier courant du script
script_dir = os.getcwd()

# === Logs ===
logs_folder = os.path.join(script_dir, "logs", pretrain_tag)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)

logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format='%(asctime)s - %(message)s'
)
logging.getLogger().addHandler(logging.StreamHandler())

# === Checkpoints ===
checkpoints_folder = os.path.join(script_dir, "checkpoints", pretrain_tag)
checkpoint_path = os.path.join(checkpoints_folder, f"model_{pretrain_tag}_best.pth")
os.makedirs(checkpoints_folder, exist_ok=True)

# Exemple de log
logging.info(f"Dossiers créés pour pré-entraînement multi-dataset : {pretrain_tag}")
logging.info(f"Checkpoint sauvegardé dans : {checkpoint_path}")

Dossiers créés pour pré-entraînement multi-dataset : pretrain_ABCD
Dossiers créés pour pré-entraînement multi-dataset : pretrain_ABCD
Dossiers créés pour pré-entraînement multi-dataset : pretrain_ABCD
Checkpoint sauvegardé dans : /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/pretrain_ABCD/model_pretrain_ABCD_best.pth
Checkpoint sauvegardé dans : /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/pretrain_ABCD/model_pretrain_ABCD_best.pth
Checkpoint sauvegardé dans : /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/pretrain_ABCD/model_pretrain_ABCD_best.pth


In [40]:
path = {
    'A' : ['datasets/A/train.json.gz', 'datasets/A/test.json.gz'],
    'B' : ['datasets/B/train.json.gz', 'datasets/B/test.json.gz'],
    'C' : ['datasets/C/train.json.gz', 'datasets/C/test.json.gz'],
    'D' : ['datasets/D/train.json.gz', 'datasets/D/test.json.gz'],
}

batch_size = 32
epochs = 5 # For each dataset. Total epochs = 4*epochs


train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

best_val_accuracy = 0


for data in path.values():
    train_path = data[0]
    test_path = data[1]

    full_dataset = GraphDataset(train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size
    
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    num_epochs = epochs  

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer, criterion, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Loading graphs from datasets/A/train.json.gz...
This may take a few minutes, please wait...


Iterating training graphs: 100%|██████████| 282/282 [00:32<00:00,  8.60batch/s]


Checkpoint saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/pretrain_ABCD/model_A_epoch_1.pth


Iterating eval graphs: 100%|██████████| 71/71 [00:04<00:00, 15.44batch/s]
Epoch 1/5, Loss: 0.9710, Train Acc: 0.4598, Val Acc: 0.3830
Epoch 1/5, Loss: 0.9710, Train Acc: 0.4598, Val Acc: 0.3830
Epoch 1/5, Loss: 0.9710, Train Acc: 0.4598, Val Acc: 0.3830


Epoch 1/5, Loss: 0.9710, Train Acc: 0.4598, Val Acc: 0.3830
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/pretrain_ABCD/model_pretrain_ABCD_best.pth


...

Epoch 4/5, Loss: 0.4797, Train Acc: 0.6970, Val Acc: 0.6975


Iterating training graphs: 100%|██████████| 257/257 [00:30<00:00,  8.41batch/s]


Checkpoint saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/pretrain_ABCD/model_A_epoch_5.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:04<00:00, 15.20batch/s]
Epoch 5/5, Loss: 0.4636, Train Acc: 0.7014, Val Acc: 0.7130
Epoch 5/5, Loss: 0.4636, Train Acc: 0.7014, Val Acc: 0.7130
Epoch 5/5, Loss: 0.4636, Train Acc: 0.7014, Val Acc: 0.7130


Epoch 5/5, Loss: 0.4636, Train Acc: 0.7014, Val Acc: 0.7130


In [41]:
torch.save(model.state_dict(), checkpoint_path)

# A

In [17]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Path to the training dataset (optional) [None]:  datasets/A/train.json.gz
Path to the test dataset [None]:  datasets/A/test.json.gz
Number of checkpoints to save during training [None]:  20
Which GPU to use if any [1]:  
GNN type (gin, gin-virtual, gcn, gcn-virtual) [gin]:  gcn-virtual
Dropout ratio [0.0]:  0.5
Number of GNN message passing layers [5]:  
Dimensionality of hidden units in GNNs [300]:  
Input batch size for training [32]:  
Number of epochs to train [10]:  400
Baseline mode: 1 (CE), 2 (Noisy CE) [1]:  2
Noise probability p (used if baseline_mode=2) [0.2]:  


Arguments received:
train_path: datasets/A/train.json.gz
test_path: datasets/A/test.json.gz
num_checkpoints: 20
device: 1
gnn: gcn-virtual
drop_ratio: 0.5
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 400
baseline_mode: 2
noise_prob: 0.2


In [18]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    criterion = GCODLoss(gamma=0.2)
else:
    criterion = torch.nn.CrossEntropyLoss()

In [19]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [20]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [21]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size
    
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0   

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer, criterion, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Loading graphs from datasets/A/train.json.gz...
This may take a few minutes, please wait...


Iterating eval graphs: 100%|██████████| 71/71 [00:04<00:00, 15.79batch/s]
Epoch 1/400, Loss: 1.2063, Train Acc: 0.2596, Val Acc: 0.3032


Epoch 1/400, Loss: 1.2063, Train Acc: 0.2596, Val Acc: 0.3032
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_A_best.pth


Iterating eval graphs: 100%|██████████| 71/71 [00:04<00:00, 15.80batch/s]
Epoch 2/400, Loss: 1.1585, Train Acc: 0.3096, Val Acc: 0.2779


Epoch 2/400, Loss: 1.1585, Train Acc: 0.3096, Val Acc: 0.2779


...

Checkpoint saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/A/model_A_epoch_200.pth


Iterating eval graphs: 100%|██████████| 71/71 [00:04<00:00, 16.19batch/s]
Epoch 200/400, Loss: 0.5730, Train Acc: 0.7055, Val Acc: 0.6653


Epoch 200/400, Loss: 0.5730, Train Acc: 0.7055, Val Acc: 0.6653


Iterating training graphs:   4%|▎         | 10/282 [00:01<00:31,  8.50batch/s]


KeyboardInterrupt: 

In [ ]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

In [ ]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

# B

In [47]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Path to the training dataset (optional) [None]:  datasets/B/train.json.gz
Path to the test dataset [None]:  datasets/B/test.json.gz
Number of checkpoints to save during training [None]:  20
Which GPU to use if any [1]:  
GNN type (gin, gin-virtual, gcn, gcn-virtual) [gin]:  gcn-virtual
Dropout ratio [0.0]:  0.5
Number of GNN message passing layers [5]:  
Dimensionality of hidden units in GNNs [300]:  
Input batch size for training [32]:  
Number of epochs to train [10]:  200
Baseline mode: 1 (CE), 2 (Noisy CE) [1]:  2
Noise probability p (used if baseline_mode=2) [0.2]:  


Arguments received:
train_path: datasets/B/train.json.gz
test_path: datasets/B/test.json.gz
num_checkpoints: 20
device: 1
gnn: gcn-virtual
drop_ratio: 0.5
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 200
baseline_mode: 2
noise_prob: 0.2


In [48]:
args.train_path = 'datasets/B/train.json.gz'
args.test_path = 'datasets/B/test.json.gz'

In [49]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3

checkpoint_path = '/home/onyxia/work/DL-Hackathon/hackaton/checkpoints/pretrain_ABCD/model_pretrain_ABCD_best.pth'
model.load_state_dict(torch.load(checkpoint_path))
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    criterion = GCODLoss(gamma=0.2)
else:
    criterion = torch.nn.CrossEntropyLoss()

In [50]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)

In [51]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [52]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size
    
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0   

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer, criterion, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Loading graphs from datasets/B/train.json.gz...
This may take a few minutes, please wait...


Iterating eval graphs: 100%|██████████| 35/35 [00:02<00:00, 16.01batch/s]
Epoch 1/200, Loss: 1.0634, Train Acc: 0.4031, Val Acc: 0.4607
Epoch 1/200, Loss: 1.0634, Train Acc: 0.4031, Val Acc: 0.4607
Epoch 1/200, Loss: 1.0634, Train Acc: 0.4031, Val Acc: 0.4607
Epoch 1/200, Loss: 1.0634, Train Acc: 0.4031, Val Acc: 0.4607
Epoch 1/200, Loss: 1.0634, Train Acc: 0.4031, Val Acc: 0.4607


Epoch 1/200, Loss: 1.0634, Train Acc: 0.4031, Val Acc: 0.4607
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_B_best.pth


Iterating eval graphs: 100%|██████████| 35/35 [00:02<00:00, 15.69batch/s]
Epoch 2/200, Loss: 0.9761, Train Acc: 0.4951, Val Acc: 0.4768
Epoch 2/200, Loss: 0.9761, Train Acc: 0.4951, Val Acc: 0.4768
Epoch 2/200, Loss: 0.9761, Train Acc: 0.4951, Val Acc: 0.4768
Epoch 2/200, Loss: 0.9761, Train Acc: 0.4951, Val Acc: 0.4768
Epoch 2/200, Loss: 0.9761, Train Acc: 0.4951, Val Acc: 0.4768


Epoch 2/200, Loss: 0.9761, Train Acc: 0.4951, Val Acc: 0.4768
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_B_best.pth


...

Epoch 52/200, Loss: 0.5710, Train Acc: 0.6929, Val Acc: 0.4661


Iterating eval graphs: 100%|██████████| 35/35 [00:02<00:00, 15.80batch/s]
Epoch 53/200, Loss: 0.5931, Train Acc: 0.6779, Val Acc: 0.4562
Epoch 53/200, Loss: 0.5931, Train Acc: 0.6779, Val Acc: 0.4562
Epoch 53/200, Loss: 0.5931, Train Acc: 0.6779, Val Acc: 0.4562
Epoch 53/200, Loss: 0.5931, Train Acc: 0.6779, Val Acc: 0.4562
Epoch 53/200, Loss: 0.5931, Train Acc: 0.6779, Val Acc: 0.4562


Epoch 53/200, Loss: 0.5931, Train Acc: 0.6779, Val Acc: 0.4562


Iterating training graphs:  86%|████████▌ | 120/140 [00:13<00:02,  8.74batch/s]


KeyboardInterrupt: 

In [53]:
torch.save(model.state_dict(), checkpoint_path)

In [34]:
plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

In [54]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

2479

In [55]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

Loading graphs from datasets/B/test.json.gz...
This may take a few minutes, please wait...


Processing graphs: 100%|██████████| 1560/1560 [00:06<00:00, 258.74graph/s]


In [56]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

Iterating eval graphs: 100%|██████████| 49/49 [00:03<00:00, 16.13batch/s]

Predictions saved to /home/onyxia/work/DL-Hackathon/hackaton/submission/testset_B.csv


# C

In [35]:
args.train_path = 'datasets/C/train.json.gz'
args.test_path = 'datasets/C/test.json.gz'

In [36]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    criterion = GCODLoss(gamma=0.2)
else:
    criterion = torch.nn.CrossEntropyLoss()

In [37]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)

In [38]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [39]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size
    
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0   

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer, criterion, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Loading graphs from datasets/C/train.json.gz...
This may take a few minutes, please wait...


Iterating eval graphs: 100%|██████████| 48/48 [00:02<00:00, 16.15batch/s]
Epoch 1/400, Loss: 1.1726, Train Acc: 0.3019, Val Acc: 0.4056
Epoch 1/400, Loss: 1.1726, Train Acc: 0.3019, Val Acc: 0.4056
Epoch 1/400, Loss: 1.1726, Train Acc: 0.3019, Val Acc: 0.4056


Epoch 1/400, Loss: 1.1726, Train Acc: 0.3019, Val Acc: 0.4056
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_C_best.pth


Iterating eval graphs: 100%|██████████| 48/48 [00:03<00:00, 14.87batch/s]
Epoch 2/400, Loss: 1.0786, Train Acc: 0.3962, Val Acc: 0.4154
Epoch 2/400, Loss: 1.0786, Train Acc: 0.3962, Val Acc: 0.4154
Epoch 2/400, Loss: 1.0786, Train Acc: 0.3962, Val Acc: 0.4154


Epoch 2/400, Loss: 1.0786, Train Acc: 0.3962, Val Acc: 0.4154
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_C_best.pth


...

Checkpoint saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/C/model_C_epoch_160.pth


Iterating eval graphs: 100%|██████████| 48/48 [00:03<00:00, 15.42batch/s]
Epoch 160/400, Loss: 0.2276, Train Acc: 0.8737, Val Acc: 0.7852
Epoch 160/400, Loss: 0.2276, Train Acc: 0.8737, Val Acc: 0.7852
Epoch 160/400, Loss: 0.2276, Train Acc: 0.8737, Val Acc: 0.7852


Epoch 160/400, Loss: 0.2276, Train Acc: 0.8737, Val Acc: 0.7852


Iterating training graphs:  88%|████████▊ | 169/192 [00:19<00:02,  8.73batch/s]


KeyboardInterrupt: 

In [40]:
plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

In [41]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

72858

In [42]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

Loading graphs from datasets/C/test.json.gz...
This may take a few minutes, please wait...


Processing graphs: 100%|██████████| 1530/1530 [00:05<00:00, 257.91graph/s]


In [43]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

Iterating eval graphs: 100%|██████████| 48/48 [00:02<00:00, 16.15batch/s]

Predictions saved to /home/onyxia/work/DL-Hackathon/hackaton/submission/testset_C.csv


# D

In [44]:
args.train_path = 'datasets/D/train.json.gz'
args.test_path = 'datasets/D/test.json.gz'

In [45]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    criterion = GCODLoss(gamma=0.2)
else:
    criterion = torch.nn.CrossEntropyLoss()

In [46]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)

In [47]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [48]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size
    
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0   

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer, criterion, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Loading graphs from datasets/D/train.json.gz...
This may take a few minutes, please wait...


Iterating eval graphs: 100%|██████████| 65/65 [00:04<00:00, 15.30batch/s]
Epoch 1/400, Loss: 1.1863, Train Acc: 0.2608, Val Acc: 0.3312
Epoch 1/400, Loss: 1.1863, Train Acc: 0.2608, Val Acc: 0.3312
Epoch 1/400, Loss: 1.1863, Train Acc: 0.2608, Val Acc: 0.3312
Epoch 1/400, Loss: 1.1863, Train Acc: 0.2608, Val Acc: 0.3312


Epoch 1/400, Loss: 1.1863, Train Acc: 0.2608, Val Acc: 0.3312
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_D_best.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:04<00:00, 15.34batch/s]
Epoch 2/400, Loss: 1.1115, Train Acc: 0.3053, Val Acc: 0.3025
Epoch 2/400, Loss: 1.1115, Train Acc: 0.3053, Val Acc: 0.3025
Epoch 2/400, Loss: 1.1115, Train Acc: 0.3053, Val Acc: 0.3025
Epoch 2/400, Loss: 1.1115, Train Acc: 0.3053, Val Acc: 0.3025


Epoch 2/400, Loss: 1.1115, Train Acc: 0.3053, Val Acc: 0.3025


...

Checkpoint saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/D/model_D_epoch_100.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:04<00:00, 14.59batch/s]
Epoch 100/400, Loss: 0.3018, Train Acc: 0.8134, Val Acc: 0.7344
Epoch 100/400, Loss: 0.3018, Train Acc: 0.8134, Val Acc: 0.7344
Epoch 100/400, Loss: 0.3018, Train Acc: 0.8134, Val Acc: 0.7344
Epoch 100/400, Loss: 0.3018, Train Acc: 0.8134, Val Acc: 0.7344


Epoch 100/400, Loss: 0.3018, Train Acc: 0.8134, Val Acc: 0.7344


Iterating training graphs:   6%|▌         | 16/257 [00:01<00:28,  8.40batch/s]


KeyboardInterrupt: 

In [49]:
plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

In [50]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

91702

In [51]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

Loading graphs from datasets/D/test.json.gz...
This may take a few minutes, please wait...


Processing graphs: 100%|██████████| 2250/2250 [00:09<00:00, 238.90graph/s]


In [52]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

Iterating eval graphs: 100%|██████████| 71/71 [00:04<00:00, 16.08batch/s]

Predictions saved to /home/onyxia/work/DL-Hackathon/hackaton/submission/testset_D.csv


In [57]:
import tarfile
import os

def gzip_folder(folder_path, output_file):
    """
    Compresses an entire folder into a single .tar.gz file.

    Args:
        folder_path (str): Path to the folder to compress.
        output_file (str): Path to the output .gz file.
    """
    with tarfile.open(output_file, "w:gz") as tar:
        tar.add(folder_path, arcname=os.path.basename(folder_path))
    print(f"Folder '{folder_path}' has been compressed into '{output_file}'")

# Example usage
folder_path = "/home/onyxia/work/DL-Hackathon/hackaton/logs"            # Path to the folder you want to compress
output_file = "/home/onyxia/work/DL-Hackathon/hackaton/logs.gz"        # Output .gz file name
gzip_folder(folder_path, output_file)

Folder '/home/onyxia/work/DL-Hackathon/hackaton/logs' has been compressed into '/home/onyxia/work/DL-Hackathon/hackaton/logs.gz'


In [58]:
# Example usage
folder_path = "/home/onyxia/work/DL-Hackathon/hackaton/submission"            # Path to the folder you want to compress
output_file = "/home/onyxia/work/DL-Hackathon/hackaton/submission.gz"        # Output .gz file name
gzip_folder(folder_path, output_file)

Folder '/home/onyxia/work/DL-Hackathon/hackaton/submission' has been compressed into '/home/onyxia/work/DL-Hackathon/hackaton/submission.gz'


In [59]:
# Example usage
folder_path = "/home/onyxia/work/DL-Hackathon/hackaton/checkpoints"            # Path to the folder you want to compress
output_file = "/home/onyxia/work/DL-Hackathon/hackaton/checkpoints.gz"        # Output .gz file name
gzip_folder(folder_path, output_file)

Folder '/home/onyxia/work/DL-Hackathon/hackaton/checkpoints' has been compressed into '/home/onyxia/work/DL-Hackathon/hackaton/checkpoints.gz'
